In [1]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv("./MEX/mex_distance_matrix.csv")
df = df.drop(['Unnamed: 0', 'shapeID'], axis = 1)
df.head(10)

,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B10,...,MEX-ADM2-1590546715-B2449,MEX-ADM2-1590546715-B2450,MEX-ADM2-1590546715-B2451,MEX-ADM2-1590546715-B2452,MEX-ADM2-1590546715-B2453,MEX-ADM2-1590546715-B2454,MEX-ADM2-1590546715-B2455,MEX-ADM2-1590546715-B2456,MEX-ADM2-1590546715-B2457,MEX-ADM2-1590546715-B2458
0,0.000000,0.000000,0.055894,0.135329,0.083422,0.000000,0.000000,0.000000,0.000000,0.373148,...,3.590395,3.301435,3.538930,3.283451,3.393920,3.570205,3.325766,3.747767,3.637447,6.987838
1,0.000000,0.000000,0.056499,0.066500,0.000000,0.025508,0.000000,0.000000,0.037331,0.271604,...,3.789347,3.477868,3.725380,3.497023,3.605621,3.772707,3.539370,3.978910,3.868678,7.211869
2,0.055894,0.056499,0.000000,0.000000,0.057884,0.000000,0.134585,0.000000,0.220951,0.231698,...,3.981246,3.672690,3.919154,3.687828,3.796559,3.963738,3.730176,4.167426,4.057289,7.401799
3,0.135329,0.066500,0.000000,0.000000,0.000000,0.083889,0.086895,0.000000,0.261674,0.000000,...,3.947168,3.622045,3.875257,3.677237,3.782247,3.933351,3.719507,4.181901,4.071220,7.400776
4,0.083422,0.000000,0.057884,0.000000,0.000000,0.141160,0.000000,0.000000,0.113149,0.000000,...,3.822413,3.504149,3.754489,3.545388,3.651453,3.807358,3.587697,4.044269,3.933669,7.267254
5,0.000000,0.025508,0.000000,0.083889,0.141160,0.000000,0.056798,0.000000,0.120956,0.346029,...,3.884624,3.585746,3.828888,3.582824,3.692719,3.865501,3.625155,4.054205,3.944155,7.291909
6,0.000000,0.000000,0.134585,0.086895,0.000000,0.056798,0.000000,0.017952,0.000000,0.000000,...,3.548168,3.226176,3.477774,3.278407,3.383091,3.534083,3.320660,3.776871,3.666266,7.000347
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017952,0.000000,0.101724,0.166607,...,3.849803,3.543507,3.788903,3.558840,3.667094,3.832473,3.601187,4.042813,3.932535,7.274808
8,0.000000,0.037331,0.220951,0.261674,0.113149,0.120956,0.000000,0.101724,0.000000,0.358514,...,3.513912,3.227151,3.463549,3.205517,3.316146,3.493420,3.247826,3.673833,3.563767,6.912072
9,0.373148,0.271604,0.231698,0.000000,0.000000,0.346029,0.000000,0.166607,0.358514,0.000000,...,3.873126,3.528537,3.789451,3.625471,3.726908,3.859831,3.667740,4.151645,4.039856,7.350714


In [8]:
df.shape

(2458, 2458)

In [9]:
def get_mins(x, num, cur_id):
    
    # Delete the current polygon from the list so we do no include it in our list of closest polgyons, then cast the dict back to a list
    x_dict = dict(x)
    del x_dict[cur_id]
    x = list(x_dict.values())
    
    # sort the distances from lowest to highest
    x_sorted = sorted(x)
    
    # If there are more polgyons that are a distance of 0 than the specified number, reutrn ALL of the indices of polygons that have a distance of zero
    if x.count(0) > num:
        zeros = x_sorted[0:x.count(0)]
        x_array = np.array(x)
        return np.where(x_array == 0)[0]
    
    # Otherwise, return the indices of the num closest polygons
    else:
        minimums = x_sorted[0:num]
        x_array = np.array(x)
        indices = [list(np.where(x_array == i)[0]) for i in minimums]
        inidces = [item for sublist in indices for item in sublist]
        res = []
        [res.append(x) for x in inidces if x not in res]
        return res

In [16]:
min_list = []
num = 8
c = 0


for col, row in df.iterrows():
    
    # Get the indices of the num closest polygons to the currect one
    mins = get_mins(row, num, df.columns[c])
    print(mins)
    
    # The get_min algo does not include the current polygon so drop that for the list of column we are going to grab from using the min indices
    ref_cols = [i for i in df.columns if i != df.columns[c]]
    
    # Get the shapeID's of the num closest polygons, append them to the overall list and increment c
    min_ids = [ref_cols[i] for i in mins]
    min_list.append(min_ids)
    c += 1
    

# Make the final data frame
min_df = pd.DataFrame(min_list)
min_df.columns = ['closest' + str(i) for i in range(0, num)]
min_df = min_df.reset_index()
min_df['index'] = df.columns#[0:5]
min_df = min_df.rename(columns = {'index':'shapeID'})
min_df.head()

[0, 4, 5, 6, 7, 611, 2299, 2300]
[0, 3, 5, 6, 4, 7, 1, 2]
[2, 4, 6, 9, 2272, 2292, 2324, 2299]
[2, 3, 6, 8, 2323, 2324, 2272, 1]
[1, 3, 5, 6, 8, 613, 2, 0]
[0, 2, 6, 2292, 2299, 9, 1, 2291]
[  0   1   4   7   8 536 571 581 613]
[0, 1, 2, 3, 4, 5, 6, 9]
[0, 6, 581, 611, 1, 536, 2300, 2311]
[3, 4, 6, 613, 2310, 2315, 2323, 571]
[2, 2272, 2292, 5, 2291, 2324, 2299, 3]
[12, 14, 11, 13, 1832, 1774, 1798, 1775]
[12, 13, 14, 11, 1832, 1774, 1798, 1775]
[11, 12, 14, 13, 1832, 1774, 1798, 1775]
[12, 14, 1832, 1831, 19, 1774, 13, 1778]
[11, 12, 13, 14, 19, 1831, 1832, 1778]
[17, 18, 1759, 1761, 16, 1758, 1757, 1751]
[18, 19, 17, 1825, 1833, 1757, 1799, 1795]
[16, 18, 17, 1757, 1758, 19, 1747, 1761]
[17, 18, 19, 1757, 1825, 15, 1833, 1796]
[15, 17, 19, 14, 1831, 1833, 1825, 1796]
[23, 1839, 1846, 1845, 1848, 195, 1834, 98]
[24, 27, 29, 30, 22, 28, 26, 2245]
[24, 25, 28, 29, 1675, 1679, 22, 1682]
[21, 24, 25, 28, 1839, 1845, 1846, 1848]
[22, 23, 24, 28, 29, 25, 27, 30]
[23, 24, 28, 1848, 25, 1838,

[200, 202, 213, 217, 223, 255, 257, 203]
[203, 213, 224, 252, 253, 258, 201, 242]
[214, 215, 216, 245, 246, 1809, 1811, 1813]
[204, 207, 220, 221, 230, 212, 210, 198]
[208, 225, 247, 256, 218, 287, 221, 212]
[ 65 204 205 207 210 220 229 232 254]
[203, 217, 244, 259, 261, 263, 239, 262]
[200, 204, 205, 231, 233, 251, 235, 202]
[198, 200, 204, 217, 233, 235, 237, 251]
[218, 238, 241, 243, 250, 263, 198, 244]
[204, 234, 251, 233, 198, 220, 217, 229]
[203, 242, 243, 262, 258, 253, 244, 245]
[200, 217, 234, 251, 233, 261, 232, 236]
[206, 211, 218, 219, 225, 235, 240, 241]
[198, 250, 259, 261, 263, 217, 218, 232]
[206, 211, 239, 1754, 1745, 219, 241, 214]
[206, 216, 235, 239, 243, 245, 214, 228]
[ 203  215  237  243  245  253  258 1767 1808]
[203 235 237 242 243 244 245 262 263]
[203, 232, 244, 259, 261, 262, 263, 217]
[215, 216, 228, 242, 243, 244, 1808, 246]
[214, 228, 1754, 1813, 206, 215, 216, 1809]
[208, 212, 221, 230, 256, 287, 225, 249]
[223, 257, 226, 255, 202, 205, 213, 217]
[199, 2

[380, 401, 423, 427, 429, 440, 391, 938]
[379, 438, 442, 2436, 2438, 2428, 2368, 425]
[380, 401, 422, 427, 437, 435, 429, 384]
[409, 415, 432, 434, 441, 443, 406, 377]
[366, 379, 442, 381, 411, 423, 368, 2436]
[365, 395, 396, 410, 431, 370, 409, 376]
[401, 422, 424, 440, 938, 976, 1054, 1338]
[371, 378, 383, 384, 397, 417, 419, 399]
[380, 391, 394, 422, 440, 428, 430, 436]
[380, 384, 394, 436, 417, 397, 430, 377]
[365, 396, 427, 443, 409, 395, 410, 370]
[399, 409, 415, 421, 425, 434, 406, 414]
[366, 382, 2427, 2337, 379, 2406, 416, 407]
[367, 406, 415, 425, 441, 433, 443, 409]
[367, 380, 406, 437, 424, 435, 401, 441]
[391, 394, 417, 431, 408, 383, 380, 384]
[367, 380, 424, 436, 401, 406, 428, 441]
[423, 442, 767, 2368, 2428, 2436, 2438, 772]
[399, 419, 415, 377, 371, 429, 390, 397]
[ 391  393  422  428  430  924  938 1000 1041]
[367, 425, 435, 443, 406, 436, 433, 409]
[366 368 369 379 423 426 439 767 779]
[365, 409, 425, 442, 435, 433, 406, 432]
[373, 385, 387, 418, 375, 396, 370, 688]

[535, 626, 648, 650, 625, 649, 647, 651]
[535, 585, 604, 625, 626, 647, 648, 541]
[533, 548, 600, 618, 558, 569, 606, 559]
[542, 588, 596, 599, 616, 632, 640, 560]
[553, 585, 632, 640, 641, 645, 596, 630]
[589, 613, 632, 645, 704, 549, 641, 588]
[541, 598, 604, 606, 618, 637, 749, 750]
[588, 613, 629, 630, 631, 640, 645, 641]
[536, 576, 586, 588, 599, 613, 560, 629]
[534, 542, 547, 560, 570, 596, 638, 616]
[544, 563, 575, 593, 639, 643, 546, 534]
[71, 73, 536, 580, 70, 586, 609, 530]
[558, 598, 618, 632, 689, 742, 749, 672]
[538, 560, 570, 583, 635, 642, 534, 592]
[544, 546, 636, 643, 651, 563, 601, 579]
[553, 596, 629, 630, 633, 585, 641, 645]
[541, 585, 630, 645, 703, 704, 641, 633]
[538, 570, 592, 639, 615, 583, 595, 560]
[539, 546, 593, 636, 640, 644, 579, 544]
[539, 546, 579, 644, 555, 574, 636, 593]
[630, 631, 633, 642, 704, 641, 589, 613]
[338, 564, 600, 619, 355, 320, 559, 345]
[548 557 578 604 606 611 627 648 649]
[626, 627, 648, 649, 650, 578, 535, 625]
[578, 648, 649, 650, 6

[833 834 839 840 846 850 862 864 869]
[824, 859, 1866, 1885, 827, 857, 822, 828]
[822, 857, 865, 828, 838, 831, 859, 830]
[823, 849, 851, 858, 870, 59, 824, 852]
[822, 830, 831, 832, 845, 846, 865, 821]
[68, 834, 835, 840, 849, 855, 833, 854]
[60, 68, 833, 835, 840, 864, 869, 834]
[825, 821, 829, 832, 856, 1865, 867, 1856]
[843, 863, 1855, 1870, 1886, 1888, 1872, 1869]
[ 843  856  860  863 1707 1708 1716 1726 1728 1869 1888]
[845, 847, 848, 852, 868, 870, 834, 853]
[831, 839, 845, 846, 847, 862, 868, 869]
[821, 832, 835, 839, 846, 861, 862, 864]
[822, 831, 845, 846, 852, 868, 847, 848]
[845, 852, 870, 868, 853, 834, 848, 855]
[53, 59, 68, 838, 840, 855, 858, 870]
[834, 835, 868, 869, 846, 833, 862, 870]
[823, 824, 838, 859, 870, 822, 852, 858]
[822, 845, 848, 849, 870, 831, 859, 852]
[834, 854, 855, 870, 868, 840, 845, 835]
[854, 855, 834, 870, 840, 850, 835, 849]
[834, 840, 850, 854, 855, 870, 849, 835]
[60, 69, 829, 844, 863, 867, 1707, 2297]
[822, 828, 837, 859, 865, 1885, 836, 1857

...

[1776, 1800, 1804, 1795, 1799, 1809, 1825, 1833]
[1768, 1785, 1790, 1792, 1808, 1793, 1794, 1811]
[224, 259, 1766, 1768, 1812, 1815, 1826, 1805]
[1773, 1775, 1778, 1780, 1797, 1828, 1763, 1798]
[1773, 1778, 1780, 1822, 1827, 1828, 1779, 1763]
[1775, 1798, 1832, 1773, 1778, 14, 1797, 1828]
[1773, 1775, 1778, 1798, 1774, 1780, 1828, 1797]
[1770 1793 1799 1800 1804 1809 1811 1821 1833]
[1765, 1781, 1786, 1788, 1789, 1830, 1801, 1818]
[1773, 1774, 1776, 1779, 1831, 1822, 1780, 1767]
[1767, 1779, 1789, 1807, 1822, 1831, 1774, 1786]
[1763, 1773, 1774, 1828, 1779, 1797, 1827, 1791]
[1765, 1767, 1778, 1789, 1801, 1818, 1830, 1788]
[1769, 1783, 1802, 1806, 1820, 1803, 1819, 1814]
[1769, 1783, 1810, 1820, 1802, 1824, 1784, 1819]
[1791, 1810, 1816, 1819, 1820, 1824, 1828, 1827]
[1768, 1771, 1790, 1808, 259, 1792, 243, 1793]
[1778, 1787, 1788, 1789, 1807, 1829, 1831, 1792]
[1787, 1823, 1829, 1831, 1790, 1764, 1792, 1833]
[1765, 1778, 1787, 1792, 1829, 1789, 1782, 1801]
[1767, 1778, 1780, 1782, 178

[1895, 1897, 1943, 1944, 1950, 1947, 1926, 1896]
[1902, 1906, 1927, 1930, 1917, 1901, 1940, 1913]
[1579, 1906, 1913, 1924, 1940, 1917, 1638, 1903]
[1608 1896 1897 1898 1902 1909 1941 1947 1950]
[1895, 1908, 1910, 1926, 1941, 1949, 1950, 1897]
[1900, 1904, 1912, 1916, 1920, 1928, 1951, 1938]
[1475, 1629, 1911, 1919, 1471, 1446, 1579, 1913]
[1897, 1944, 1941, 1902, 1895, 1896, 1925, 1908]
[1457, 1556, 1911, 1912, 1933, 1952, 1594, 1481]
[1899 1904 1907 1908 1910 1914 1916 1921 1945]
[1896 1901 1910 1915 1923 1926 1931 1934 1939 1941 1944 1945]
[1904, 1916, 1946, 1928, 1907, 1910, 1950, 1938]
[1594, 1911, 1933, 1949, 1556, 1629, 1481, 1446]
[474, 506, 1494, 1921, 1936, 2392, 2416, 2421]
[1968, 1982, 1988, 2028, 2059, 2064, 2142, 2036]
[1551, 1620, 2106, 2125, 1562, 1503, 1545, 2096]
[1980, 2043, 2073, 2094, 2154, 2045, 1994, 2061]
[1960, 2086, 2104, 2110, 2151, 2006, 2022, 2129]
[1968, 2029, 2048, 2051, 2061, 2143, 2005, 1967]
[2044, 2074, 2138, 2148, 1972, 2038, 2052, 2118]
[1958, 2006, 

[2051, 2072, 2164, 2124, 1988, 2161, 2022, 2084]
[1970, 2019, 2069, 2129, 1971, 2135, 1539, 2111]
[2079, 2110, 2139, 2165, 2010, 2043, 2117, 1966]
[1970, 1971, 2069, 2111, 2128, 2151, 2047, 2157]
[1980, 2081, 2133, 2160, 1982, 1972, 2082, 1990]
[1987, 2005, 2115, 2122, 2106, 2004, 1974, 2156]
[2002, 2006, 2046, 2103, 2072, 2095, 2062, 2049]
[1972, 1980, 1982, 1990, 2034, 2082, 2131, 2160]
[1964, 1999, 2004, 2005, 2083, 2156, 2094, 2007]
[1539, 1593, 2019, 2055, 2061, 2069, 1613, 2128]
[2029, 2060, 2086, 2088, 2091, 2155, 1955, 2037]
[1155, 2070, 2074, 975, 1009, 2101, 2154, 2159]
[1960, 1973, 2045, 2075, 2146, 2053, 2032, 2031]
[2010, 2041, 2079, 2110, 2121, 2129, 2117, 2013]
[2020, 2038, 2073, 1489, 2064, 2071, 2001, 2009]
[459, 465, 517, 528, 2014, 496, 1976, 452]
[1955, 1969, 2052, 2060, 2078, 2143, 1989, 2065]
[1959, 1967, 1968, 1969, 1989, 2003, 2030, 2143]
[1988, 2011, 2036, 2040, 2161, 2164, 2092, 2077]
[1998, 2045, 2058, 2099, 2112, 2119, 2148, 2125]
[1973, 2031, 2032, 2045, 21

[69, 857, 1708, 2279, 844, 1717, 2317, 1702]
[2280, 2285, 2291, 2297, 2320, 2325, 2290, 2301]
[0, 5, 2284, 2293, 2300, 2311, 2292, 2282]
[0, 612, 2300, 2311, 8, 5, 2284, 6]
[2280, 2285, 2297, 2314, 2327, 2290, 2299, 2292]
[578, 601, 2275, 2307, 2313, 2322, 2329, 652]
[2289, 2312, 2320, 2295, 2306, 2277, 2308, 2318]
[565, 572, 622, 2275, 2278, 2294, 2322, 2328]
[581, 2276, 2310, 2315, 2316, 2323, 577, 2328]
[294, 305, 314, 2281, 2290, 2312, 2317, 2277]
[546, 2275, 2303, 2313, 2326, 2329, 2294, 2322]
[ 284  561  566  806 2283 2289 2318 2319 2320 2325]
[296, 2289, 2318, 290, 298, 284, 2309, 2320]
[9, 572, 614, 2306, 2315, 2316, 2328, 2305]
[ 352  612 1696 1706 1710 1725 1730 2282 2284 2300 2301]
[2277, 2281, 2289, 2290, 2295, 2304, 2307, 2320]
[546, 601, 2303, 2308, 2326, 2322, 652, 578]
[1722, 2280, 2285, 2287, 2292, 2302, 2324, 2327]
[9, 2306, 2311, 2316, 2323, 2328, 2276, 581]
[571, 2276, 2286, 2296, 2306, 2311, 2316, 2328]
[  47   69  314 1702 1707 1708 2274 2279 2290 2307]
[284, 296,

ValueError: Length mismatch: Expected axis has 22 elements, new values have 8 elements

In [24]:
# Make the final data frame
min_df = pd.DataFrame(min_list)
min_df = min_df.drop([8,9,10,11,12,13,14,15,16,17,18,19,20,21], axis = 1)
min_df.columns = ['closest' + str(i) for i in range(0, num)]
min_df = min_df.reset_index()
min_df['index'] = df.columns#[0:5]
min_df = min_df.rename(columns = {'index':'shapeID'})
min_df.head()

,shapeID,closest0,closest1,closest2,closest3,closest4,closest5,closest6,closest7
0,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B613,MEX-ADM2-1590546715-B2301,MEX-ADM2-1590546715-B2302
1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4
2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2294,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2301
3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B2325,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2
4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B615,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B1


In [25]:
min_df.to_csv("./MEX/closest_polygons.csv", index = False)

In [26]:
min_df.head()

,shapeID,closest0,closest1,closest2,closest3,closest4,closest5,closest6,closest7
0,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B613,MEX-ADM2-1590546715-B2301,MEX-ADM2-1590546715-B2302
1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4
2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2294,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2301
3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B2325,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2
4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B615,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B1


In [27]:
min_df.shape

(2458, 9)